In [1]:
library(Seurat)
library(dplyr)
library(ggplot2)
library(DirichletReg)

set.seed(47)
setwd("~/Dropbox (MIT)/Zambia/")
options(repr.plot.width = 8, repr.plot.height = 8)
source("helper_scripts/plot_cluster_meta_percentage.R")
library(RColorBrewer)
cell_color_scheme = c(brewer.pal(n = 8, name = "Set2"),brewer.pal(n = 9, name = "Set1"),brewer.pal(n = 8, name = "Set3"))
# patient_color_scheme = readRDS("color_palette/cell_color_scheme.rdds")
setwd("~/Dropbox (MIT)/Zambia/reseq_analysis/")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Formula
Loading required package: rgl


In [5]:
combined = readRDS("combined_iterate_jan13.rds")

In [6]:
comb_zambia = subset(combined,study%in%c("EE"))
comb_zambia$region = factor(comb_zambia$region, levels = c("Duodenum", "Bulb", "Jejunum"))

Fischer's exact leave-one out testing still makes the most sense to me as the way to go

In [7]:
hiv_colors = c("#9c954d","#b067a3")
disease_colors = readRDS("disease_colors.rds")
region_colors = readRDS("region_colors.rds")

In [8]:
Idents(comb_zambia) <- "tier4"
s_obj = subset(comb_zambia,HIV.HTLV.=="N")
s_obj$cell_types = s_obj$tier4
all_counts = generate_all_counts(s_obj@meta.data)

sample_all_counts = generate_counts_by_sample(all_counts)
sample_meta = generate_sample_meta(s_obj@meta.data)
sample_meta = sample_meta[order(sample_meta$orig.ident),]
sample_all_counts = cbind(sample_all_counts, region=sample_meta$region)

fischer_bulb = run_fischer(s_obj,"region","Bulb",s_obj$tier4)
fischer_duodenum = run_fischer(s_obj,"region","Duodenum",s_obj$tier4)
fischer_jejunem = run_fischer(s_obj,"region","Jejunum",s_obj$tier4)

pvals_bulb = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(pvals_bulb) <- fischer_bulb$clusters
pvals_duodenum = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(pvals_duodenum) <- fischer_duodenum$clusters
pvals_jejunem = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(pvals_jejunem) <- fischer_jejunem$clusters

signs_bulb = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(signs_bulb) <- fischer_bulb$clusters
signs_duodenum = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(signs_duodenum) <- fischer_duodenum$clusters
signs_jejunem = matrix(rep(1,nrow(sample_all_counts)*length(unique(s_obj$cell_types))),
                  nrow=nrow(sample_all_counts),
                  ncol=length(unique(s_obj$cell_types)))
colnames(signs_jejunem) <- fischer_jejunem$clusters

for(i in 1:length(unique(s_obj$orig.ident))){
    print(i)
    temp = subset(s_obj, orig.ident!=unique(s_obj$orig.ident)[i])

    pval_table_bulb = run_fischer(temp,"region","Bulb",temp$tier4)
    pval_table_duodenum = run_fischer(temp,"region","Duodenum",temp$tier4)
    pval_table_jejunem = run_fischer(temp,"region","Jejunum",temp$tier4)

    sign_table_bulb = run_fischer(temp,"region","Bulb",temp$tier4)
    sign_table_duodenum = run_fischer(temp,"region","Duodenum",temp$tier4)
    sign_table_jejunem = run_fischer(temp,"region","Jejunum",temp$tier4)

    for(j in 1:nrow(pval_table_bulb)){
        pvals_bulb[i,colnames(pvals_bulb)==pval_table_bulb$clusters[j]] = pval_table_bulb$adj_pval[j]
        pvals_duodenum[i,colnames(pvals_duodenum)==pval_table_duodenum$clusters[j]] = pval_table_duodenum$adj_pval[j]
        pvals_jejunem[i,colnames(pvals_jejunem)==pval_table_jejunem$clusters[j]] = pval_table_jejunem$adj_pval[j]

        signs_bulb[i,colnames(signs_bulb)==sign_table_bulb$clusters[j]] = sign_table_bulb$sign[j]
        signs_duodenum[i,colnames(signs_duodenum)==sign_table_duodenum$clusters[j]] = sign_table_duodenum$sign[j]
        signs_jejunem[i,colnames(signs_jejunem)==sign_table_jejunem$clusters[j]] = sign_table_jejunem$sign[j]

    }
  
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19


In [9]:
same_sign = function(x){
    total = sum(x > 0)
    if(total ==0 ){
        return(TRUE)
        
    }else if(total==length(x)){
        return(TRUE)
    }
    else{
        return(FALSE)
    }
}
get_max_pvals = function(pvals,signs){
    max_pvals = apply(pvals,2,max)
    same_signs = apply(signs,2,same_sign)
    max_pvals = max_pvals[same_signs]
    return(data.frame(max_adj_pvals=max_pvals))
}

max_pvals_bulb = get_max_pvals(pvals_bulb,signs_bulb)
max_pvals_bulb$cell_types = rownames(max_pvals_bulb)
max_pvals_duodenum = get_max_pvals(pvals_duodenum,signs_duodenum)
max_pvals_duodenum$cell_types = rownames(max_pvals_duodenum)
max_pvals_jejunum = get_max_pvals(pvals_jejunem,signs_jejunem)
max_pvals_jejunum$cell_types = rownames(max_pvals_jejunum)

In [33]:
saveRDS(max_pvals_bulb,"composition/hiv_neg_only_bulb_jan19.rds")
saveRDS(max_pvals_duodenum,"composition/hiv_neg_only_duodenum_jan19.rds")
saveRDS(max_pvals_jejunum,"composition/hiv_neg_only_jejunum_jan19.rds")


In [10]:
max_pvals_bulb = readRDS("composition/hiv_neg_only_bulb_jan19.rds")
max_pvals_duodenum = readRDS("composition/hiv_neg_only_duodenum_jan19.rds")
max_pvals_jejunum = readRDS("composition/hiv_neg_only_jejunum_jan19.rds")


In [18]:
max_pvals_bulb %>% filter(max_adj_pvals< 0.05)

max_adj_pvals,cell_types
<dbl>,<chr>
1.599563e-02,BEST4
4.833588e-02,Endothelial
1.007273e-03,Ent SI
1.844110e-02,Epi DPCR1 TFF1
1.030953e-05,Epi FABP1 CD55
2.373984e-05,Epi LYZ TFF2
1.985419e-02,Goblet
5.134115e-18,Stem OLFM4
4.744092e-06,T CD4 CD69lo


In [11]:
distinct_tier4 = distinct_at(comb_zambia@meta.data,vars(tier1,tier4))

In [12]:
fischer_jejunum = fischer_jejunem
sig_bulb = max_pvals_bulb %>% filter(max_adj_pvals < 0.05)
fischer_bulb$cell_types = fischer_bulb$clusters
enriched_bulb = left_join(sig_bulb,fischer_bulb) %>% filter(sign==1)

sig_duodenum = max_pvals_duodenum %>% filter(max_adj_pvals < 0.05)
fischer_duodenum$cell_types = fischer_duodenum$clusters
enriched_duodenum = left_join(sig_duodenum,fischer_duodenum) %>% filter(sign==1)

sig_jejunum = max_pvals_jejunum %>% filter(max_adj_pvals < 0.05)
fischer_jejunum$cell_types = fischer_jejunum$clusters
enriched_jejunum = left_join(sig_jejunum,fischer_jejunum) %>% filter(sign==1)

get_region = function(orig){
    reg = substr(orig,nchar(orig),nchar(orig)+1)
    if(reg=="B"){
        return("Bulb")
    }
    if(reg=="D"){
        return("Duodenum")
    }
    if(reg=="J"){
        return("Jejunum")
    }
}
regions = sapply(all_counts$orig.ident,get_region)
all_counts$region = regions

bulb_cells = enriched_bulb$cell_types
duodenum_cells = enriched_duodenum$cell_types
jejunum_cells = enriched_jejunum$cell_types

bulb_cells_unique = bulb_cells[!(bulb_cells %in% c(duodenum_cells,jejunum_cells))]
duodenum_cells_unique = duodenum_cells[!(duodenum_cells %in% c(bulb_cells,jejunum_cells))]
jejunum_cells_unique = jejunum_cells[!(jejunum_cells %in% c(bulb_cells,duodenum_cells))]

Joining, by = "cell_types"Warning message:
“Column `cell_types` joining character vector and factor, coercing into character vector”Joining, by = "cell_types"Warning message:
“Column `cell_types` joining character vector and factor, coercing into character vector”Joining, by = "cell_types"Warning message:
“Column `cell_types` joining character vector and factor, coercing into character vector”

In [13]:
enriched_bulb

max_adj_pvals,cell_types,clusters,adj_pval,sign
<dbl>,<chr>,<fct>,<dbl>,<dbl>
4.833588e-02,Endothelial,Endothelial,2.807658e-07,1
1.125749e-07,T gamma delta CXCR4hi,T gamma delta CXCR4hi,4.042928e-15,1
1.077651e-04,T gamma delta GZMAhi,T gamma delta GZMAhi,6.191928e-21,1


In [ ]:
pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_bulb_enriched.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% bulb_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_blank(),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Duodenal bulb enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()

pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_bulb_enrichedxlab.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% bulb_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_text(angle = 45, hjust = 1),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Duodenal bulb enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()

pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_duodenum_enriched.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% duodenum_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_blank(),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Distal duodenum enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()

pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_duodenum_enrichedxlab.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% duodenum_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_text(angle = 45, hjust = 1),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Distal duodenum enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()

pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_jejunum_enriched.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% jejunum_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_blank(),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Jejunum enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()

pdf("~/zambia_eed_figures/supp_stricture_region_hiv/hiv_neg_region_jejunum_enrichedxlab.pdf",useDingbats = F,height=6)

ggplot(all_counts %>% filter(cell_types %in% jejunum_cells_unique),aes(x=cell_types,y=percent_of_sample,fill=region)) + geom_boxplot() +
   # geom_dotplot(binaxis='y', stackdir='center', dotsize=0.3,position=position_dodge(0.8)) +
    theme_classic() +
    theme(text = element_text(size=20),axis.text.x = element_text(angle = 45, hjust = 1),  panel.border = element_blank(),  
  # Remove panel grid lines
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.position="top",
  # Remove panel background
  panel.background = element_blank()) +
    ggtitle("Jejunum enriched cell types") + xlab("") + ylab("Fraction of all cells in sample") +scale_fill_manual(values=region_colors) 

dev.off()